In [ ]:
import numpy as np
import pandas as pd

In [ ]:
books = pd.read_csv('Books.csv')
users = pd.read_csv('Users.csv')
ratings = pd.read_csv('Ratings.csv')

In [ ]:
books.head()

In [ ]:
print(books.shape)

In [ ]:
print(ratings.shape)

In [ ]:
books.isnull().sum()

In [ ]:
ratings.isnull().sum()

In [ ]:
books.duplicated()

Popularity based

In [ ]:
ratings

In [ ]:
ratings_with_name = ratings.merge(books, on='ISBN')

In [ ]:
ratings_with_name

In [ ]:
num_rating_data_frame = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_data_frame.rename(columns={'Book-Rating':'num_rating'},inplace=True)
# Joining the data frames on book titles
num_rating_data_frame

In [ ]:
avg_rating_data_frame = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_data_frame.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avg_rating_data_frame

In [ ]:
popular_df = num_rating_data_frame.merge(avg_rating_data_frame,on='Book-Title')
popular_df      

In [ ]:
popular_df = popular_df[popular_df['num_rating']>=250].sort_values('avg_rating',ascending=False).head(50)

In [ ]:
popular_df

In [ ]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_rating','avg_rating']]

Collaborative filtering  with a matrix factorization approach


In [ ]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
nice_users = x[x].index #boolean indexing

In [ ]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(nice_users)]

In [ ]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50
famous_books = y[y].index
famous_books

In [ ]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [ ]:
final_ratings.drop_duplicates()

In [ ]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [ ]:
pt.fillna(0,inplace=True)
pt

In [ ]:
from sklearn.metrics.pairwise  import cosine_similarity

In [ ]:
similarity_scores = cosine_similarity(pt)
similarity_scores[0]

In [ ]:
similarity_scores.shape

In [ ]:
def recommend(book_name):
    index = np.where(pt.index == book_name)[0][0]   # get the row index
    similar_items = sorted(list(enumerate(similarity_scores[index])), key = lambda x: x[1], reverse=True)[1:6]
    
    data = []
    for i in similar_items:
        items = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        items.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        items.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        items.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(items)
    return data

In [ ]:
recommend('The Da Vinci Code')

In [ ]:
popular_df

In [ ]:
popular_df['Image-URL-M'][0]

In [ ]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))

In [ ]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity.pkl','wb'))